# Converting the PDF to Text

The PDFs are converted to text using this package: https://github.com/jsvine/pdfplumber#extracting-text. Follow the installation instructions before running this notebook. 


In [8]:
import re 
import pdfplumber

full_pdf_text = ""

# input the path to the PDF you want to convert here
with pdfplumber.open("foucault.pdf") as pdf:

    for i in range(len(pdf.pages)):
        full_page = pdf.pages[i].extract_text()

        first_line = full_page.split("\n")[0] # where page numbers at the top of the page are likely to be found
        last_line = full_page.split("\n")[-1] # where page numbers at the bottom of the page are likely to be found
        full_pdf_text += full_page

        if len(full_page) != 0:
            top_left_page_matches = re.findall("^([xXvViI]+|\d+)\s", first_line)
            top_right_page_matches = re.findall("\s([xXvViI]+|\d+)\r$", first_line)
            bottom_page_matches = re.findall("([xXvViI]+|\d+)$", last_line)
            
            all_matches = []
            if top_left_page_matches:
                all_matches.append(top_left_page_matches[0].strip())
            if top_right_page_matches:
                all_matches.append(top_right_page_matches[-1].strip())
            if bottom_page_matches:
                all_matches.append(bottom_page_matches[-1].strip())

            if len(all_matches) > 0:
                if len(all_matches) > 1: # more than one potential page number 
                    print(f'\n{len(all_matches)} potential page numbers were found for this page.')
                    print(all_matches)
                    print(f'Check PDF page {i+1} for the correct page number and enter it below (exactly as displayed in the print) If the page has no printed page number, do not input anything.')
                    correct_page_number = input()
                    print(correct_page_number)
                    if len(correct_page_number.strip()) != 0:
                        printed_page_number = correct_page_number.strip()
                    else:
                        printed_page_number = None
                else:
                    printed_page_number = all_matches[0]

                if printed_page_number is not None: 
                    full_pdf_text += f'\n\n~printed_page_number:{printed_page_number}~'
                
        full_pdf_text += f'\n~indexed_page_number:{i+1}~\n\n'

# enter the name of your output file here     
with open("gender_trouble_output.txt", "w") as text_file:
    text_file.write(full_pdf_text)



2 potential page numbers were found for this page.
['vi', '233']
Check PDF page 7 for the correct page number and enter it below (exactly as displayed in the print) If the page has no printed page number, do not input anything.
vi

2 potential page numbers were found for this page.
['212', 'I']
Check PDF page 213 for the correct page number and enter it below (exactly as displayed in the print) If the page has no printed page number, do not input anything.
212

2 potential page numbers were found for this page.
['234', '155']
Check PDF page 233 for the correct page number and enter it below (exactly as displayed in the print) If the page has no printed page number, do not input anything.
234

2 potential page numbers were found for this page.
['235', '13']
Check PDF page 234 for the correct page number and enter it below (exactly as displayed in the print) If the page has no printed page number, do not input anything.
235

2 potential page numbers were found for this page.
['236', '10